In [1]:
import pandas as pd
import numpy as np

In [2]:
# Mounting the colab on the Google drive
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
%cd /content/drive/MyDrive/Data\ Mining\ Project\ Research/Data

/content/drive/.shortcut-targets-by-id/1uKZmpRRwciHwohqfjKcRePjwzyJ7Qyhg/Data Mining Project Research/Data


In [2]:
import pandas as pd
import random
dfinal1 = pd.read_csv('Final_Data.csv')
dfinal2 = dfinal1.dropna(subset = ['Overall_tag_DNS'])


In [3]:
dfinal3 = dfinal2[['body','Overall_tag_DNS']]

In [4]:
train = pd.DataFrame(dfinal3.body)
test = pd.DataFrame(dfinal3.Overall_tag_DNS)

In [5]:
test

,Overall_tag_DNS
256,D
257,D
258,D
421,N
477,D
...,...
48743,D
48747,D
48749,S
48765,S


In [6]:
from sklearn.model_selection import cross_validate, train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, test,
    test_size=0.2, shuffle = True, random_state = 8)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
    test_size=0.25, random_state= 8) # 0.25 x 0.8 = 0.2


print("X_train shape: {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_train shape: {}".format(y_train.shape))
print("y_test shape: {}".format(y_test.shape))
print("X_val shape: {}".format(X_val.shape))
print("y val shape: {}".format(y_test.shape))

X_train shape: (9577, 1)
X_test shape: (3193, 1)
y_train shape: (9577, 1)
y_test shape: (3193, 1)
X_val shape: (3193, 1)
y val shape: (3193, 1)


In [7]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X=dfinal3.drop('Overall_tag_DNS',axis=1)
Y=le.fit_transform(dfinal3['Overall_tag_DNS'])

a=dict(zip(le.inverse_transform([0,1,2]),[0,1,2]))
a

{'D': 0, 'N': 1, 'S': 2}

In [8]:
y_test['Overall_tag_DNS']=y_test['Overall_tag_DNS'].map(a)
y_train['Overall_tag_DNS']=y_train['Overall_tag_DNS'].map(a)
y_val['Overall_tag_DNS']=y_val['Overall_tag_DNS'].map(a)

C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
y_train['Overall_tag_DNS'].value_counts()

0    5725
2    2142
1    1710
Name: Overall_tag_DNS, dtype: int64

In [22]:
!pip install tensorflow==2.2

  Using cached google_auth-1.35.0-py2.py3-none-any.whl (152 kB)
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.3.3
    Uninstalling google-auth-2.3.3:
      Successfully uninstalled google-auth-2.3.3
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0
    Uninstalling tensorflow-2.7.0:
      Successfully uninstalled tensorflow-2.7.0


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\hp\\anaconda3\\lib\\site-packages\\~ensorflow\\lite\\experimental\\microfrontend\\python\\ops\\_audio_microfrontend_op.so'
Consider using the `--user` option or check the permissions.



In [10]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [11]:
reviews = X_train['body']
y= y_train['Overall_tag_DNS']

def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

tokenized_reviews = [tokenize_reviews(review) for review in reviews]

y = np.array(y)



In [15]:
reviews_with_len = [[review, y[i], len(review)] for i, review in enumerate(tokenized_reviews)]
reviews_with_len.sort(key=lambda x: x[2])
BATCH_SIZE = 64
sorted_reviews_labels = [(review_lab[0], review_lab[1], review_lab[2]) for review_lab in reviews_with_len]
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32,tf.int32))
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ),(), ()))
OUTPUT_CLASSES = 3



In [16]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=3,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [17]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 3
DROPOUT_RATE = 0.2

NB_EPOCHS = 4

text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

2.7.0


In [18]:
text_model.fit(batched_dataset, epochs=NB_EPOCHS)

Epoch 1/4


InternalError:  Failed to call dnnl_sgemm. Error code: 2
	 [[node gradient_tape/text_model/conv1d_2/Conv1D/Conv2DBackpropInput
 (defined at C:\Users\HP\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:464)
]] [Op:__inference_train_function_22957]

Errors may have originated from an input operation.
Input Source operations connected to node gradient_tape/text_model/conv1d_2/Conv1D/Conv2DBackpropInput:
In[0] gradient_tape/text_model/conv1d_2/Conv1D/ShapeN:	
In[1] text_model/conv1d_2/Conv1D/ExpandDims_1 (defined at C:\Users\HP\anaconda3\lib\site-packages\keras\layers\convolutional.py:238)	
In[2] gradient_tape/text_model/conv1d_2/Conv1D/Reshape:

Operation defined at: (most recent call last)
>>>   File "C:\Users\HP\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
>>>     "__main__", mod_spec)
>>> 
>>>   File "C:\Users\HP\anaconda3\lib\runpy.py", line 85, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "C:\Users\HP\anaconda3\lib\site-packages\ipykernel_la

In [11]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X=dfinal3.drop('Overall_tag_DNS',axis=1)
Y=le.fit_transform(dfinal3['Overall_tag_DNS'])

In [12]:
a=dict(zip(le.inverse_transform([0,1,2]),[0,1,2]))
a

{'D': 0, 'N': 1, 'S': 2}

In [331]:
dfinal3['Overall_tag_DNS']=dfinal3['Overall_tag_DNS'].map(a)

In [21]:
!pip install bert-for-tf2
!pip install sentencepiece

In [29]:
!pip install tensorflow

In [193]:
!pip install tensorflow_hub

In [22]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [30]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [333]:
# reviews = dfinal3['body']
# y= dfinal3['Overall_tag_DNS']

In [334]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [335]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]


In [336]:

y = np.array(y)


In [337]:
import numpy as np

In [338]:
reviews_with_len = [[review, y[i], len(review)] for i, review in enumerate(tokenized_reviews)]
# max_size = 2402
# reviews_with_len = [[[review[cnt] if cnt < len(review) else 0 for cnt in range(max_size)], y[i], len(review)]
# for i, review in enumerate(tokenized_reviews)]



In [339]:
import random
random.shuffle(reviews_with_len)

In [340]:
reviews_with_len.sort(key=lambda x: x[2])

In [370]:
BATCH_SIZE = 50

sorted_reviews_labels = [(review_lab[0], review_lab[1], review_lab[2]) for review_lab in reviews_with_len]
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32,tf.int32))
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ),(), ()))
OUTPUT_CLASSES = 3


In [371]:
# sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]
#sorted_reviews_labels = [(review_lab[0], review_lab[1], review_lab[2]) for review_lab in reviews_with_len]


In [372]:
#processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))
#processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32,tf.int32))

In [373]:
#batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

#batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ),(), ()))


In [374]:
next(iter(batched_dataset))


(<tf.Tensor: shape=(50, 2), dtype=int32, numpy=
 array([[ 6149,     0],
        [ 2959,     0],
        [ 2748,     0],
        [ 1029,     0],
        [ 3599,     0],
        [ 2748,     0],
        [ 2053,     0],
        [ 2206,     0],
        [27937,     0],
        [ 7078,     0],
        [  100,     0],
        [15624,     0],
        [ 4283,     0],
        [10392,     0],
        [ 6149,     0],
        [ 2053,     0],
        [ 6149,     0],
        [ 4283,     0],
        [ 2748,     0],
        [ 2771,     0],
        [  100,     0],
        [ 2748,     0],
        [ 2053,     0],
        [ 2748,     0],
        [ 2053,     0],
        [ 3530,     0],
        [ 3658,     0],
        [ 2053,     0],
        [ 2053,     0],
        [ 2053,  1012],
        [ 2053,  1012],
        [ 2748,  1012],
        [ 4283,   999],
        [ 2003,  8257],
        [17404,  1029],
        [ 2589,  1012],
        [ 5262,  1012],
        [ 9616,  1029],
        [ 6429,   999],
        [ 2117, 

In [ ]:
from sklearn.model_selection import cross_validate, train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=False)

In [375]:

def get_dataset_partitions_tf(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    #train_ds = ds.take(train_size)  
    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [376]:
# import math
# TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
# TEST_BATCHES = TOTAL_BATCHES // 10
# batched_dataset.shuffle(TOTAL_BATCHES)

# test_data = batched_dataset.skip(TOTAL_BATCHES -TEST_BATCHES)
# train_data = batched_dataset.skip(TEST_BATCHES)


#train_data, validation_data, test_data =get_dataset_partitions_tf(batched_dataset,260)

In [377]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=3,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [378]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 3
DROPOUT_RATE = 0.2

NB_EPOCHS = 6

In [379]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [380]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [381]:
# sorted_reviews_labels = [(review_lab[0], review_lab[1], review_lab[2]) for review_lab in reviews_with_len]
# processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32,tf.int32))
# batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ),(), ()))
# OUTPUT_CLASSES = 3

In [384]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/6
    116/Unknown - 20s 162ms/step - loss: 70.5040 - sparse_categorical_accuracy: 0.5876

InvalidArgumentError:  Computed output size would be negative: -1 [input_size: 2, effective_filter_size: 4, stride: 1]
	 [[node text_model/conv1d_14/Conv1D
 (defined at C:\Users\HP\anaconda3\lib\site-packages\keras\layers\convolutional.py:238)
]] [Op:__inference_train_function_126642]

Errors may have originated from an input operation.
Input Source operations connected to node text_model/conv1d_14/Conv1D:
In[0] text_model/conv1d_14/Conv1D/ExpandDims:	
In[1] text_model/conv1d_14/Conv1D/ExpandDims_1:

Operation defined at: (most recent call last)
>>>   File "C:\Users\HP\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
>>>     "__main__", mod_spec)
>>> 
>>>   File "C:\Users\HP\anaconda3\lib\runpy.py", line 85, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
>>>     app.launch_new_instance()
>>> 
>>>   File "C:\Users\HP\anaconda3\lib\site-packages\traitlets\config\application.py", l

In [8]:

text_model1 = tf.keras.models.load_model('./saved_model')

In [273]:
results = text_model.evaluate(test_data)

40/40 [==============================] - 2s 30ms/step - loss: 0.0433 - sparse_categorical_accuracy: 1.0000


In [23]:
text_model.predict(train_data,verbose=True)

NameError: ignored

In [29]:
results = text_model.evaluate(test_data)


49/49 [==============================] - 8s 101ms/step - loss: 0.0324 - sparse_categorical_accuracy: 1.0000


In [30]:
!pip install pyyaml h5py

In [36]:
#tf.saved_model.save(text_model,export_dir = './saved_model')

INFO:tensorflow:Assets written to: ./saved_model/assets


INFO:tensorflow:Assets written to: ./saved_model/assets


In [32]:
export_dir = './saved_model'
tf.keras.models.save_model(text_model,filepath = export_dir)

INFO:tensorflow:Assets written to: ./saved_model/assets


INFO:tensorflow:Assets written to: ./saved_model/assets


In [12]:

text_model1 = tf.keras.models.load_model('./saved_model')
#text_model1  =tf.saved_model.load(export_dir)


In [13]:
text_model1

In [57]:
text_model

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt


In [44]:
a = text_model.predict(train_data,verbose=True)


450/450 [==============================] - 19s 41ms/step


NameError: ignored

In [41]:
l1=[]
for i in a:
    l1.append(np.argmax(i))


In [45]:
a

array([[5.62385190e-04, 9.99019623e-01, 4.18011303e-04],
       [3.93844880e-02, 9.19233799e-01, 4.13817056e-02],
       [9.86874774e-02, 2.04498202e-01, 6.96814299e-01],
       ...,
       [1.00000000e+00, 4.98861751e-11, 3.07547268e-08],
       [1.00000000e+00, 1.36769086e-11, 5.86125459e-08],
       [1.00000000e+00, 1.82745789e-12, 1.56455360e-09]], dtype=float32)

In [89]:
l1

[2,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 1,
 2,
 1,
 1,
 1,
 0,
 2,
 1,
 2,
 2,
 2,
 0,
 0,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 1,
 1,
 2,
 2,
 1,
 0,
 2,
 0,
 1,
 2,
 0,
 0,
 2,
 1,
 2,
 2,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 2,
 1,
 0,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 0,
 1,
 0,
 0,
 1,
 2,
 1,
 0,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 2,
 1,
 2,
 1,
 0,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 0,
 2,
 1,
 0,
 0,
 0,
 2,
 1,
 2,
 1,
 1,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 0,
 1,
 1,
 2,
 2,
 0,
 0,
 0,
 2,
 1,
 2,
 0,
 0,
 1,
 2,
 1,
 2,
 0,
 1,
 1,
 2,
 0,
 2,
 0,
 1,
 0,
 2,
 2,
 1,
 0,
 2,
 1,
 1,
 1,
 2,
 0,
 2,
 2,
 1,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 1,
 0,
 0,
 0,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 2,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 0,
 0,
 0,
 1,
 2,
 2,
 0,
 1,
 2,


In [9]:
dfinal_na =dfinal1[dfinal1.Overall_tag_DNS.isna()]
dfinal_na['Overall_tag_DNS'] = 4

C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
dfinal_na

,author,body,Index1,author_flair_text,created_utc,id,Comment_or_submission,Overall_tag_DNS
0,lillygirl0528,i thought this was a conspiracy group? why is ...,143.0,NaN,1.636751e+09,hkdo3du,C,4
1,ktyn,I got Moderna.\n\nFirst dose: sore arm for a f...,274.0,NaN,1.627998e+09,h7jvuhf,C,4
2,ktyn,The company I work for is also giving $100 to ...,347.0,NaN,1.620936e+09,gy0o0uz,C,4
3,ktyn,"I’ve had the same thoughts, glad to see I’m no...",380.0,NaN,1.616941e+09,gskvnfr,C,4
4,dvncan90,90 days with or without vaccination? I don't h...,622.0,NaN,1.632317e+09,hdubc6u,C,4
...,...,...,...,...,...,...,...,...
48774,iamapremed,Hi everyone!\n\nSo I'm working on reapplicatio...,NaN,NaN,1.590077e+09,gnzf48,S,4
48775,dontwannabeabadger,"Hi all,\n\nI am a biology major at a big ten s...",NaN,NaN,1.588386e+09,gby1j6,S,4
48776,ubcthrowaway-01,I have a 4.5 gpa on a 4.33 (I got over 100% in...,NaN,NaN,1.625639e+09,ofd778,S,4
48777,taelea,"Hi all, \n\nSo I have my UBC interview coming ...",NaN,NaN,1.612451e+09,lchidk,S,4


In [11]:
dfinal_na.shape

(32816, 8)

In [12]:
dfinal_na_full=dfinal_na

In [164]:
dfinal_na=dfinal_na_full[32000:]

In [165]:
tokenized_reviews = [tokenize_reviews(review) for review in dfinal_na['body']]


In [166]:
y = np.array(dfinal_na['Overall_tag_DNS'])

In [167]:
reviews_with_len = [[review, y[i],len(review)] for i, review in enumerate(tokenized_reviews)]

In [168]:
reviews_with_len.sort(key=lambda x: x[2])

In [169]:
len(reviews_with_len)

816

In [170]:
BATCH_SIZE = 32

sorted_reviews_labels = [(review_lab[0], review_lab[1],review_lab[2]) for review_lab in reviews_with_len]
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32,tf.int32,tf.int32))
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), (),()))
#OUTPUT_CLASSES = 3

In [171]:
import math
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10

#test_data = batched_dataset.skip(TOTAL_BATCHES -TEST_BATCHES)
#train_data = batched_dataset.skip(TEST_BATCHES)#

In [172]:

a= text_model1.predict(batched_dataset,verbose=True)

26/26 [==============================] - 1s 28ms/step


In [173]:
#l = []
for i in a:
    l.append(np.argmax(i))


In [40]:
pd.DataFrame(l).to_csv("12000_tak.csv",index =False)

In [158]:
l = []
for i in range(0,len(dfinal_na_full)-1000,1000):
    print(i)
    dfinal_na=dfinal_na_full[i:i+1000]
    tokenized_reviews = [tokenize_reviews(review) for review in dfinal_na['body']]
    y = np.array(dfinal_na['Overall_tag_DNS'])
    reviews_with_len = [[review, y[i],len(review)] for i, review in enumerate(tokenized_reviews)]
    reviews_with_len.sort(key=lambda x: x[2])
    BATCH_SIZE = 32

    sorted_reviews_labels = [(review_lab[0], review_lab[1],review_lab[2]) for review_lab in reviews_with_len]
    processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32,tf.int32,tf.int32))
    batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), (),()))
    #OUTPUT_CLASSES = 3
    
    a= text_model1.predict(batched_dataset,verbose=True)
   
    for i in a:
        l.append(np.argmax(i))


    
    
    

0
32/32 [==============================] - 1s 17ms/step
1000
32/32 [==============================] - 1s 17ms/step
2000
32/32 [==============================] - 0s 14ms/step
3000
32/32 [==============================] - 1s 15ms/step
4000
32/32 [==============================] - 0s 15ms/step
5000
32/32 [==============================] - 0s 14ms/step
6000
32/32 [==============================] - 1s 16ms/step
7000
32/32 [==============================] - 1s 20ms/step
8000
32/32 [==============================] - 0s 13ms/step
9000
32/32 [==============================] - 0s 14ms/step
10000
32/32 [==============================] - 0s 15ms/step
11000
32/32 [==============================] - 0s 14ms/step
12000
32/32 [==============================] - 1s 23ms/step
13000
32/32 [==============================] - 1s 15ms/step
14000
32/32 [==============================] - 1s 18ms/step
15000
32/32 [==============================] - 0s 14ms/step
16000
32/32 [==============================] - 0s 12m

In [174]:
len(l)

32816

In [175]:
c1 =c2=c3= 0
for i in l:
    
    if(i==0):
        c1 +=1
    elif(i==1):
        c2+=1
    else:
        c3+=1
    

In [176]:
c1, c2,c3

(11401, 14003, 7412)

In [231]:
test_pred_res = []
test_pred= text_model1.predict(test_data,verbose=True)   
for i in test_pred:
    test_pred_res.append(np.argmax(i))

49/49 [==============================] - 3s 29ms/step


In [232]:
len(test_pred_res)

1563

In [212]:
comb_pred = []
for i in test_data:
    comb_pred.append(list(i[1].numpy()))

In [225]:
test_actual = [item for sublist in comb_pred for item in sublist]


In [226]:
len(test_actual)

1563

In [233]:
counter =0
for i in range(len(test_actual)):
    if(test_actual[i] == test_pred_res[i]):
        counter+=1
acc = counter / len(test_actual)

In [236]:
c1 =c2=c3= 0
for i in test_actual:
    
    if(i==0):
        c1 +=1
    elif(i==1):
        c2+=1
    else:
        c3+=1

In [237]:
c1,c2,c3

(1161, 169, 233)

In [238]:
final_tag = np.array(l)

In [239]:
final_tag

array([1, 1, 2, ..., 0, 0, 0], dtype=int64)

In [245]:
df1 = dfinal1

In [246]:
df1['Overall_tag_DNS'][dfinal1.Overall_tag_DNS.isna()] = final_tag

C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [247]:
df1

,author,body,Index1,author_flair_text,created_utc,id,Comment_or_submission,Overall_tag_DNS
0,lillygirl0528,i thought this was a conspiracy group? why is ...,143.0,NaN,1.636751e+09,hkdo3du,C,1
1,ktyn,I got Moderna.\n\nFirst dose: sore arm for a f...,274.0,NaN,1.627998e+09,h7jvuhf,C,1
2,ktyn,The company I work for is also giving $100 to ...,347.0,NaN,1.620936e+09,gy0o0uz,C,2
3,ktyn,"I’ve had the same thoughts, glad to see I’m no...",380.0,NaN,1.616941e+09,gskvnfr,C,2
4,dvncan90,90 days with or without vaccination? I don't h...,622.0,NaN,1.632317e+09,hdubc6u,C,1
...,...,...,...,...,...,...,...,...
48774,iamapremed,Hi everyone!\n\nSo I'm working on reapplicatio...,NaN,NaN,1.590077e+09,gnzf48,S,0
48775,dontwannabeabadger,"Hi all,\n\nI am a biology major at a big ten s...",NaN,NaN,1.588386e+09,gby1j6,S,0
48776,ubcthrowaway-01,I have a 4.5 gpa on a 4.33 (I got over 100% in...,NaN,NaN,1.625639e+09,ofd778,S,0
48777,taelea,"Hi all, \n\nSo I have my UBC interview coming ...",NaN,NaN,1.612451e+09,lchidk,S,0


In [248]:
df1.to_csv('Final_data_with_DNStagging.csv',index = False)

In [300]:
dfinal5 = dfinal2[['body','Overall_tag_DNS']][13000:]
dfinal6=dfinal5.copy()

In [301]:
len(dfinal5)

2963

NameError: name 'dfinal_full_na' is not defined

In [283]:
l = []
for i in range(0,len(dfinal_na_full)-1000,100):
    print(i)
    dfinal_na=dfinal5[i:i+100]
    tokenized_reviews = [tokenize_reviews(review) for review in dfinal_na['body']]
    y = np.array(dfinal_na['Overall_tag_DNS'])
    reviews_with_len = [[review, y[i],len(review)] for i, review in enumerate(tokenized_reviews)]
    reviews_with_len.sort(key=lambda x: x[2])
    BATCH_SIZE = 32

    sorted_reviews_labels = [(review_lab[0], review_lab[1],review_lab[2]) for review_lab in reviews_with_len]
    processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32,tf.int32,tf.int32))
    batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), (),()))
    #OUTPUT_CLASSES = 3
    
    a= text_model1.predict(batched_dataset,verbose=True)
   
    for i in a:
        l.append(np.argmax(i))


    
    
    

0
4/4 [==============================] - 0s 17ms/step
100
4/4 [==============================] - 0s 21ms/step
200
4/4 [==============================] - 0s 25ms/step
300
4/4 [==============================] - 0s 27ms/step
400
4/4 [==============================] - 0s 18ms/step
500
4/4 [==============================] - 0s 26ms/step
600
4/4 [==============================] - 0s 22ms/step
700
4/4 [==============================] - 0s 27ms/step
800
4/4 [==============================] - 0s 20ms/step
900
4/4 [==============================] - 0s 32ms/step
1000
4/4 [==============================] - 0s 20ms/step
1100
4/4 [==============================] - 0s 22ms/step
1200
4/4 [==============================] - 1s 377ms/step
1300
4/4 [==============================] - 0s 26ms/step
1400
4/4 [==============================] - 0s 29ms/step
1500
4/4 [==============================] - 0s 27ms/step
1600
4/4 [==============================] - 0s 25ms/step
1700
4/4 [==============================] 

ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [287]:
test_pred=l

In [309]:
test_actual=dfinal6['Overall_tag_DNS']

In [310]:
test_actual=list(test_actual)

TypeError: 'numpy.ndarray' object is not callable

In [311]:
counter =0
for i in range(len(test_actual)):
    if(test_actual[i] == test_pred[i]):
        counter+=1
acc = counter / len(test_actual)

In [312]:
acc

0.4333445831927101

In [299]:
dfinal6

,body,Overall_tag_DNS
36123,Thank you for these posts! Wish we learned mor...,4
36124,I’m pretty sure I commented last year about th...,4
36127,Thanks for the post!\n\nFor the direct real es...,4
36128,Thank you so much for this! Graduating school ...,4
36130,"Gosh, as a soon to be med student with the onl...",4
...,...,...
48743,"Hi all,\n\nI am a pharmacy student who applied...",4
48747,Saw this morning that Russia has apparently ap...,4
48749,I had a sore throat and mild headache today so...,4
48765,I just got the second dose of Pfizer and... I ...,4


In [308]:
dfinal6['Overall_tag_DNS']=dfinal6['Overall_tag_DNS'].map(a)